<a href="https://colab.research.google.com/github/ch0uchou/Resnet50-MNIST/blob/main/ResNet_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model Resnet50

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, ZeroPadding2D, MaxPooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import GlobalAveragePooling2D

# Load MNIST data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Preprocess the data
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0
# Expand the dimensions of the data to (28, 28, 1) to fit the ConvNet
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)
# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)
# Define the identity block
def identity_block(X, f, filters, stage, block):
    # Retrieve Filters
    F1, F2, F3 = filters
    # Save the input value
    X_shortcut = X
    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name='res' + str(stage) + block + '_branch2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2a')(X)
    X = Activation('relu')(X)
    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same',
               name='res' + str(stage) + block + '_branch2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2b')(X)
    X = Activation('relu')(X)
    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name='res' + str(stage) + block + '_branch2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2c')(X)
    # Add shortcut value to main path
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X
# Define the convolutional block
def convolutional_block(X, f, filters, stage, block, s=2):
    # Retrieve Filters
    F1, F2, F3 = filters
    # Save the input value
    X_shortcut = X
    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s), padding='valid',
               name='res' + str(stage) + block + '_branch2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2a')(X)
    X = Activation('relu')(X)
    # Second component of main path
    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same',
               name='res' + str(stage) + block + '_branch2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2b')(X)
    X = Activation('relu')(X)
    # Third component of main path
    X = Conv2D(F3, (1, 1), strides=(1, 1), padding='valid',
               name='res' + str(stage) + block + '_branch2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch2c')(X)
    # Shortcut Path
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s), padding='valid',
                        name='res' + str(stage) + block + '_branch1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name='bn' + str(stage) + block + '_branch1')(X_shortcut)
    # Add shortcut value to main path
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X
# Define the ResNet50 model adjusted for the MNIST dataset
def ResNet50(input_shape=(28, 28, 1), classes=10):
    # Define the input tensor
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(1, 1), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(1, 1))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL . Use "X = AveragePooling2D(...)(X)"
    X = GlobalAveragePooling2D()(X)

    # output layer
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    return model
# Instantiate the model
model = ResNet50(input_shape=(28, 28, 1), classes=10)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Fit the model on the data
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))
# Save the model
model.save('mnist_resnet50.h5')
# Output to indicate save completion
print("Model saved as mnist_resnet50.h5")


11501568/11490434 [==============================] - 9s 1us/step
Epoch 1/10
1875/1875 [==============================] - 217s 111ms/step - loss: 0.1903 - accuracy: 0.9494 - val_loss: 0.0868 - val_accuracy: 0.9787
Epoch 2/10
1875/1875 [==============================] - 207s 111ms/step - loss: 0.0736 - accuracy: 0.9792 - val_loss: 0.2869 - val_accuracy: 0.9320
Epoch 3/10
1875/1875 [==============================] - 207s 111ms/step - loss: 0.0587 - accuracy: 0.9828 - val_loss: 0.1177 - val_accuracy: 0.9702
Epoch 4/10
1875/1875 [==============================] - 207s 110ms/step - loss: 0.0525 - accuracy: 0.9844 - val_loss: 0.0450 - val_accuracy: 0.9854
Epoch 5/10
1875/1875 [==============================] - 207s 110ms/step - loss: 0.0417 - accuracy: 0.9872 - val_loss: 0.0486 - val_accuracy: 0.9859
Epoch 6/10
1875/1875 [==============================] - 208s 111ms/step - loss: 0.0369 - accuracy: 0.9890 - val_loss: 0.0381 - val_accuracy: 0.9877
Epoch 7/10
1875/1875 [=========================

d:\1-programme\porgramme\anaconda\envs\test1\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Model saved as mnist_resnet50.h5


In [3]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load the model
model = load_model('mnist_resnet50.h5')

# Load the MNIST dataset
(_, _), (test_images, test_labels) = mnist.load_data()

# Preprocess the data
test_images = test_images.astype('float32') / 255.0
test_images = np.expand_dims(test_images, axis=-1)  # Expand the dimensions to (28, 28, 1) to fit the ConvNet
test_labels = to_categorical(test_labels, 10)

# Select a single test image and its label
index = 0  # You can change this to test other images
test_image = test_images[index]
test_label = test_labels[index]

# Add batch dimension (model expects (batch_size, height, width, channels))
test_image = np.expand_dims(test_image, axis=0)

# Predict
prediction = model.predict(test_image)

# Output prediction
predicted_class = np.argmax(prediction, axis=1)
actual_class = np.argmax(test_label)

print(f'Predicted class: {predicted_class[0]}')
print(f'Actual class: {actual_class}')


Predicted class: 7
Actual class: 7
